# [**Exemple de pipeline pour l'extraction et l'évaluation des caractéristiques de texte**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_text_feature_extraction.ipynb)<br/>([*Sample pipeline for text feature extraction and evaluation*](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_text_feature_extraction.html))

Le jeu de données utilisé dans cet exemple est [**le jeu de données de textes 20 newsgroups** (7.2.2)](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset) qui sera automatiquement téléchargé, mis en cache et réutilisé pour l'exemple de classification de documents.

Dans cet exemple, nous réglons les hyperparamètres d'un classificateur particulier en utilisant un [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Pour une démonstration sur les performances de certains autres classificateurs, consultez le notebook [**Classification des documents de textes à l'aide de caractéristiques creuses**](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html).

In [1]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Mathieu Blondel <mathieu@mblondel.org>
#         Arturo Amor <david-arturo.amor-quiroz@inria.fr>
# License: BSD 3 clause

# Chargement des données

Nous chargeons deux catégories du jeu d'entraînement. Vous pouvez ajuster le nombre de catégories en ajoutant leurs noms à la liste ou en définissant `categories=None` lors de l'appel du chargeur de jeu de données `fetch20newsgroups` pour obtenir les 20 d'entre elles.

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    "alt.atheism",
    "talk.religion.misc",
]

data_train = fetch_20newsgroups(
    subset="train",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=("headers", "footers", "quotes"),
)

data_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=("headers", "footers", "quotes"),
)

print(f"Loading 20 newsgroups dataset for {len(data_train.target_names)} categories:")
print(data_train.target_names)
print(f"{len(data_train.data)} documents")

Loading 20 newsgroups dataset for 2 categories:
['alt.atheism', 'talk.religion.misc']
857 documents


# Pipeline avec ajustement des hyperparamètres

Nous définissons un pipeline combinant un vectoriseur de caractéristiques de texte avec un classificateur simple mais efficace pour la classification de textes.

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", ComplementNB()),
    ]
)
pipeline

Pipeline(steps=[('vect', TfidfVectorizer()), ('clf', ComplementNB())])

Nous définissons une grille d'hyperparamètres à explorer par [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). En utilisant plutôt [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), toutes les combinaisons possibles sur la grille seraient explorées, ce qui peut être coûteux à calculer, alors que le paramètre `n_iter` de [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) contrôle le nombre de combinaisons aléatoires différentes qui sont évaluées. Veuillez noter que définir `n_iter` plus grand que le nombre de combinaisons possibles dans une grille conduirait à répéter des combinaisons déjà explorées. Nous recherchons la meilleure combinaison de paramètres pour l'extraction de caractéristiques (`vect__`) et pour le classificateur (`clf__`).

In [3]:
import numpy as np

parameter_grid = {
    "vect__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
    "vect__min_df": (1, 3, 5, 10),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
    "vect__norm": ("l1", "l2"),
    "clf__alpha": np.logspace(-6, 6, 13),
}

Dans ce cas, `n_iter=40` n'est pas une recherche exhaustive de la grille d'hyperparamètres. En pratique, il serait intéressant d'augmenter le paramètre `n_iter` pour obtenir une analyse plus informative. En conséquence, le temps de calcul augmente. Nous pouvons le réduire en profitant de la parallélisation sur l'évaluation des combinaisons de paramètres en augmentant le nombre de CPUs utilisés via le paramètre `n_jobs`.

In [4]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=parameter_grid,
    n_iter=40,
    random_state=0,
    n_jobs=2,
    verbose=1,
)

print("Performing grid search...")
print("Hyperparameters to be evaluated:")
pprint(parameter_grid)

Performing grid search...
Hyperparameters to be evaluated:
{'clf__alpha': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
 'vect__max_df': (0.2, 0.4, 0.6, 0.8, 1.0),
 'vect__min_df': (1, 3, 5, 10),
 'vect__ngram_range': ((1, 1), (1, 2)),
 'vect__norm': ('l1', 'l2')}


In [5]:
from time import time

t0 = time()
random_search.fit(data_train.data, data_train.target)
print(f"Done in {time() - t0:.3f}s")

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Done in 44.419s


In [6]:
print("Best parameters combination found:")
best_parameters = random_search.best_estimator_.get_params()
for param_name in sorted(parameter_grid.keys()):
    print(f"{param_name}: {best_parameters[param_name]}")

Best parameters combination found:
clf__alpha: 0.01
vect__max_df: 0.2
vect__min_df: 1
vect__ngram_range: (1, 1)
vect__norm: l1


In [7]:
test_accuracy = random_search.score(data_test.data, data_test.target)
print(
    "Accuracy of the best parameters using the inner CV of "
    f"the random search: {random_search.best_score_:.3f}"
)
print(f"Accuracy on test set: {test_accuracy:.3f}")

Accuracy of the best parameters using the inner CV of the random search: 0.816
Accuracy on test set: 0.709


Les préfixes `vect` et `clf` sont nécessaires pour éviter les ambiguïtés possibles dans le pipeline, mais ne sont pas nécessaires pour visualiser les résultats. Pour cette raison, nous définissons une fonction qui renommera les hyperparamètres ajustés et améliorera la lisibilité.

In [8]:
import pandas as pd


def shorten_param(param_name):
    """Remove components' prefixes in param_name."""
    return param_name.rsplit("__", 1)[1] if "__" in param_name else param_name


cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results.rename(shorten_param, axis=1)

Nous pouvons utiliser un [`plotly.express.scatter`](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html) pour visualiser le compromis entre le temps de notation et le score moyen de test (c'est-à-dire le "CV score"). En passant le curseur sur un point donné, nous affichons les paramètres correspondants. Les barres d'erreur correspondent à une déviation standard telle que calculée dans les différents plis de la validation croisée.

In [9]:
import plotly.express as px

param_names = [shorten_param(name) for name in parameter_grid.keys()]
labels = {
    "mean_score_time": "CV Score time (s)",
    "mean_test_score": "CV score (accuracy)",
}
fig = px.scatter(
    cv_results,
    x="mean_score_time",
    y="mean_test_score",
    error_x="std_score_time",
    error_y="std_test_score",
    hover_data=param_names,
    labels=labels,
)
fig.update_layout(
    title={
        "text": "trade-off between scoring time and mean test score",
        "y": 0.95,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    }
)
fig

Notez que les clusters de modèles dans le coin supérieur gauche du graphique ont le meilleur compromis entre précision et temps de scoring. Dans ce cas, l'utilisation de bigrammes augmente le temps de scoring requis sans considérablement améliorer la précision du pipeline.

**Note** : pour plus d'informations sur la façon de personnaliser un réglage automatisé pour maximiser le score et minimiser le temps de scoring, voir le notebook d'exemple [**Stratégie de réajustement personnalisée d'une recherche en grille avec validation croisée**](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html).

Nous pouvons également utiliser un [`plotly.express.parallel_coordinates`](https://plotly.com/python-api-reference/generated/plotly.express.parallel_coordinates.html) pour visualiser davantage la moyenne du score de test en fonction des hyperparamètres ajustés. Cela aide à trouver les interactions entre plus de deux hyperparamètres et à fournir une intuition sur leur pertinence pour améliorer les performances d'un pipeline.

Nous appliquons une transformation `math.log10` sur l'axe `alpha` pour étendre la plage active et améliorer la lisibilité du graphique. Une valeur sur cet axe doit être comprise comme $10^x$.

In [10]:
import math

column_results = param_names + ["mean_test_score", "mean_score_time"]

transform_funcs = dict.fromkeys(column_results, lambda x: x)
# Using a logarithmic scale for alpha
transform_funcs["alpha"] = math.log10
# L1 norms are mapped to index 1, and L2 norms to index 2
transform_funcs["norm"] = lambda x: 2 if x == "l2" else 1
# Unigrams are mapped to index 1 and bigrams to index 2
transform_funcs["ngram_range"] = lambda x: x[1]

fig = px.parallel_coordinates(
    cv_results[column_results].apply(transform_funcs),
    color="mean_test_score",
    color_continuous_scale=px.colors.sequential.Viridis_r,
    labels=labels,
)
fig.update_layout(
    title={
        "text": "Parallel coordinates plot of text classifier pipeline",
        "y": 0.99,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    }
)
fig

Le graphique de coordonnées parallèles affiche les valeurs des hyperparamètres sur différentes colonnes, tandis que la métrique de performance est codée en couleur. Il est possible de sélectionner une plage de résultats en cliquant et en maintenant enfoncé n'importe quel axe du graphique de coordonnées parallèles. Vous pouvez ensuite faire glisser (déplacer) la sélection de plage et couper deux sélections pour voir les intersections. Vous pouvez annuler une sélection en cliquant à nouveau sur le même axe.

En particulier pour cette recherche d'hyperparamètres, il est intéressant de remarquer que les meilleurs modèles ne semblent pas dépendre de la norme de régularisation, mais dépendent d'un compromis entre `max_df`, `min_df` et la force de régularisation `alpha`. La raison en est que l'inclusion de caractéristiques bruyantes (c'est-à-dire `max_df` proche de 1.0 ou `min_df` proche de 0) tend à surapprendre et nécessite donc une régularisation plus forte pour compenser. Avoir moins de caractéristiques nécessite moins de régularisation et moins de temps de scoring.

Les meilleurs scores de précision sont obtenus lorsque `alpha` est compris entre $10^{-6}$ et $10^0$, indépendamment de l'hyperparamètre `norm`.